## Бизнес-процесс
Клиент оформляет заявку на сайте банка на какой-либо продукт. \
Сайт отправляет заявку в CRM-систему, которая присваивает свой внутренний id заявке и запускает процесс кредитного скоринга \
и выдачи продукта в случае положительного решения для клиента.

### Описание данных
В приложении к заданию дан набор данных в файле **dataset.xlsx**. \
Листы SITE и CRM являются таблицами в базе данных. Таблица SITE содержит заявки, сформированные на сайте. \
Таблица CRM содержит id заявки в CRM-системе, id заявки в системе сайта, id клиента, продукт, \
на который оформлена заявка, решение (одобрено / нет) и наличие факта выдачи продукта. 

### Задания
#### Задание 1.
Напишите SQL-запрос, который бы возвращал таблицу, на основе которой можно построить воронку продаж от количества заявок на сайте до выдачи с группировкой по продукту.\
Выдвиньте гипотезу о причине расхождений в количестве заявок между системой сайта и crm-системой.

#### Задание 2.
Клиенты могут совершать повторные заявки. Характеристики клиентов, которые делают повторные заявки и которые их не делают, отличаются. Необходимо отсегментировать заявки из таблицы CRM исходя из того, сколько всего клиент сделал заявок. Напишите SQL-запрос, который бы рассчитывал дополнительное поле для таблицы CRM, где находился бы сегмент клиента по признаку количества сделанных заявок (шаг для сегментации задайте произвольно).
Рассмотрите воронку продаж в разрезе полученных сегментов.
Какие закономерности это позволило выявить? Предложите объяснение.

#### Задание 3.
Напишите sql-запрос, который бы исходя из данных таблицы CRM рассчитывал бы таблицу, содержащую три поля, где id клиента являлось бы уникальным значением, а также признаки «клиент делал заявку на кредит» и «клиент делал заявку на кредитную карту» (1 – делал, неважно сколько раз, 0 – не делал).

#### Задание 4.
Напишите SQL-запрос, который исходя из данных таблицы CRM возвращал бы предыдущую заявку клиента на этот же продукт для текущей заявки (например, если текущая заявка – на кредитную карту, то в поле должно содержаться id предыдущей заявки на кредитную карту, а не на кредит).
Результат должен содержать поля: APPLICATION_ID, CLIENT_ID, ASK_PROD_TYPE_NM, INTEGRATION_ID, PREV_APPLICATION_ID (предыдущая заявка)


In [1]:
import sqlite3
import pandas as pd

In [2]:
# прочитать с помощью Pandas данные из excel
data_site = pd.read_excel('dataset.xlsx', sheet_name='SITE')
data_crm = pd.read_excel('dataset.xlsx', sheet_name='CRM')

In [3]:
data_site.head()

,APPLICATION_ID
0,SITE-10
1,SITE-1
2,SITE-4
3,SITE-22
4,SITE-11


In [4]:
data_crm.head()

,APPLICATION_ID,CLIENT_ID,INTEGRATION_ID,ASK_PROD_TYPE_NM,APPROVED_FLG,ISSUED_FLG
0,1,52363,SITE-10,Кредит,1,0
1,2,69440,SITE-1,Кредитная карта,0,0
2,3,69387,SITE-4,Кредитная карта,0,0
3,4,36849,SITE-22,Кредит,1,0
4,5,69487,SITE-11,Кредитная карта,0,0


In [5]:
#проверка на дубли в APPLICATION_ID, INTEGRATION_ID
duplicate_check = data_crm.duplicated(subset=['APPLICATION_ID', 'INTEGRATION_ID']).any()
if duplicate_check:
    print("Есть несколько записей с одним APPLICATION_ID или INTEGRATION_ID")
else:
    print("Критичных дублей нет")

Критичных дублей нет


In [6]:
#заливаем в базу
with sqlite3.connect('db') as conn:
    data_site.to_sql(name='site', con=conn, index=False, if_exists='replace')
    data_crm.to_sql(name='crm', con=conn, index=False, if_exists='replace')


Задание 1. Напишите SQL-запрос, который бы возвращал таблицу, на основе которой можно построить воронку продаж от количества заявок на сайте до выдачи с группировкой по продукту. Выдвиньте гипотезу о причине расхождений в количестве заявок между системой сайта и crm-системой.

In [7]:
conn = sqlite3.connect('db')

In [8]:
pd.read_sql("""
SELECT 
     c.ASK_PROD_TYPE_NM AS product
    ,COUNT(DISTINCT s.APPLICATION_ID) as sites_count
    ,COUNT(DISTINCT c.APPLICATION_ID) as crm_count
    ,SUM(c.APPROVED_FLG) AS approved
    ,SUM(c.ISSUED_FLG) AS issued
FROM site AS s
LEFT JOIN crm AS c ON s.APPLICATION_ID = c.INTEGRATION_ID
GROUP BY c.ASK_PROD_TYPE_NM
ORDER BY c.ASK_PROD_TYPE_NM
""", conn)

,product,sites_count,crm_count,approved,issued
0,None,1312,0,NaN,NaN
1,Кредит,63731,63731,8597.0,1350.0
2,Кредитная карта,66106,66106,11900.0,4210.0


Как видно из таблицы выше, 1312 заявок, оставленных на сайте, ещё не появились в crm-системе.
Такое может быть по разным причинам. Например, с лагом между попаданием их с сайта в crm, или с тем, что во время заполнения произошёл интеграционный сбой и часть пакетов просто с данными по этим заявкам просто не попали в crm-систему, или же эти заявки в принципе не должны попадать в конкретную CRM.

Задание 2
Клиенты могут совершать повторные заявки. Характеристики клиентов, которые делают повторные заявки и которые их не делают, отличаются. Необходимо отсегментировать заявки из таблицы CRM исходя из того, сколько всего клиент сделал заявок. Напишите SQL-запрос, который бы рассчитывал дополнительное поле для таблицы CRM, где находился бы сегмент клиента по признаку количества сделанных заявок (шаг для сегментации задайте произвольно). Рассмотрите воронку продаж в разрезе полученных сегментов. Какие закономерности это позволило выявить? Предложите объяснение.

In [9]:
pd.read_sql("""
SELECT DISTINCT
   product
   ,segment
   ,SUM(sites_count) as sites_count
   ,SUM(crm_count) as crm_count
   ,SUM(approved) as approved
   ,100 * SUM(approved) / SUM(crm_count) AS '%req/approve'
   ,SUM(issued) as issued
   ,100 * SUM(issued) / SUM(approved) AS '%approve/issued'
   ,100 * SUM(issued) / SUM(crm_count) AS '%req/issued'
FROM (
   SELECT
      c.ASK_PROD_TYPE_NM AS product
      ,CASE 
         WHEN COUNT(DISTINCT c.APPLICATION_ID) = 1 THEN '1'
         WHEN COUNT(DISTINCT c.APPLICATION_ID) BETWEEN 2 AND 5 THEN '1-5'
         WHEN COUNT(DISTINCT c.APPLICATION_ID) BETWEEN 6 AND 10 THEN '5-10'
         ELSE '>10' END AS segment
      ,COUNT(DISTINCT s.APPLICATION_ID) AS sites_count
      ,COUNT(DISTINCT c.APPLICATION_ID) AS crm_count
      ,SUM(c.APPROVED_FLG) AS approved
      ,SUM(c.ISSUED_FLG) AS issued
   FROM site AS s
   LEFT JOIN crm AS c ON s.APPLICATION_ID = c.INTEGRATION_ID
   WHERE c.ASK_PROD_TYPE_NM IS NOT NULL
   GROUP BY c.ASK_PROD_TYPE_NM, c.CLIENT_ID
) AS subquery
GROUP BY product, segment;
""", conn)

,product,segment,sites_count,crm_count,approved,%req/approve,issued,%approve/issued,%req/issued
0,Кредит,1,50568,50568,7267,14,1145,15.0,2
1,Кредит,1-5,12828,12828,1303,10,199,15.0,1
2,Кредит,5-10,262,262,27,10,6,22.0,2
3,Кредит,>10,73,73,0,0,0,NaN,0
4,Кредитная карта,1,47753,47753,9613,20,3563,37.0,7
5,Кредитная карта,1-5,17982,17982,2279,12,641,28.0,3
6,Кредитная карта,5-10,300,300,7,2,5,71.0,1
7,Кредитная карта,>10,71,71,1,1,1,100.0,1


Первая закономерность, которую мы видим - чем больше было подано заявок, тем меньший из них % одобрения.
Это может бытьт связано с тем, что люди подавали заявки по нескольку раз, в надежде, что на сей раз она будет одобрена, но этого не происходило. Аналогичная история с отношением факта выдачи продукта к общему числу заявок.

Также мы видим, что те, у кого было более 5 заявок, чаще получали продукт. Это может быть связано с тем, что те, у кого было мало заявок вплоть до одной, зачастую отказывались от кредита и просто не получали договор на руки, в то время как те, кому этот крежит был нужен, с большей вероятностью получают впоследтвии продукт на руки.
Однако те, у кого было от 2 до 5 заявок, получали продукт на руки реже, чем те, у кого была 1 заявка. Вероятно, потому что одновременно с этим у них были заявки и в других банках, так как они продолжали поиск предложений, получив первый отказ здесь, а когда кредит был одобрен у нас в банке, наше предложение им стало уже неактуально.

Также из воронки видно, что на кредитные карты заявки одобряются чаще, чем на кредиты наличными. Вероятно, из-за более смягченных условий, таких как минимальная сумма кредита.

При этом, на кредит наличными первичную заявку оставило больше людей, чем на кредитную карту. Возможно, сработала таргетная реклама, т.к. на кредиты идёт более активное привлечение, так как они выгоднее нам (банку).

Также мы видим, что среди тех, кто оставил более 10 заявок на кредит наличными, никому не одобрили ни одну заявку.

Задание 3
Напишите sql-запрос, который бы исходя из данных таблицы CRM рассчитывал бы таблицу, содержащую три поля, где id клиента являлось бы уникальным значением, а также признаки «клиент делал заявку на кредит» и «клиент делал заявку на кредитную карту» (1 – делал, неважно сколько раз, 0 – не делал).

In [10]:
pd.read_sql('''
SELECT
     CLIENT_ID
    ,MAX(CASE WHEN ASK_PROD_TYPE_NM = 'Кредит' THEN 1 ELSE 0 END) AS credit_req_flag
    ,MAX(CASE WHEN ASK_PROD_TYPE_NM = 'Кредитная карта' THEN 1 ELSE 0 END) AS ccard_req_flag
FROM crm
GROUP BY CLIENT_ID
''', conn)

,CLIENT_ID,credit_req_flag,ccard_req_flag
0,1,1,0
1,2,0,1
2,3,1,0
3,4,1,0
4,5,0,1
...,...,...,...
107861,110061,0,1
107862,110062,0,1
107863,110063,1,0
107864,110064,0,1


Задание 4
Напишите SQL-запрос, который исходя из данных таблицы CRM возвращал бы предыдущую заявку клиента на этот же продукт для текущей заявки (например, если текущая заявка – на кредитную карту, то в поле должно содержаться id предыдущей заявки на кредитную карту, а не на кредит). Результат должен содержать четыре поля: APPLICATION_ID, CLIENT_ID, ASK_PROD_TYPE_NM, INTEGRATION_ID, PREV_APPLICATION_ID (предыдущая заявка)

In [11]:
pd.read_sql('''
WITH prev_apps AS (
    SELECT
         APPLICATION_ID
        ,CLIENT_ID
        ,ASK_PROD_TYPE_NM
        ,INTEGRATION_ID
        ,LAG(INTEGRATION_ID) OVER (PARTITION BY CLIENT_ID, ASK_PROD_TYPE_NM ORDER BY APPLICATION_ID) AS PREV_APPLICATION_ID
    FROM crm
)
SELECT DISTINCT
     pa.APPLICATION_ID
    ,pa.CLIENT_ID
    ,pa.ASK_PROD_TYPE_NM
    ,pa.PREV_APPLICATION_ID
FROM prev_apps AS pa
INNER JOIN (SELECT 
                CLIENT_ID
               ,ASK_PROD_TYPE_NM
               ,COUNT(*) AS APP_CNT 
            FROM crm
            GROUP BY CLIENT_ID, ASK_PROD_TYPE_NM) AS ca 
ON ca.CLIENT_ID = pa.CLIENT_ID AND ca.ASK_PROD_TYPE_NM = pa.ASK_PROD_TYPE_NM
WHERE ca.APP_CNT > 1
''', conn)

,APPLICATION_ID,CLIENT_ID,ASK_PROD_TYPE_NM,PREV_APPLICATION_ID
0,13065,28,Кредит,None
1,109010,28,Кредит,SITE-13306
2,17987,36,Кредитная карта,None
3,18008,36,Кредитная карта,SITE-18334
4,46815,42,Кредит,None
...,...,...,...,...
31511,15472,110058,Кредит,SITE-419
31512,22087,110058,Кредит,SITE-15760
31513,107381,110058,Кредит,SITE-22510
31514,107286,110058,Кредитная карта,None
